# Sabermetrics: The Air Quality Factor

In this Jupyter notebook I will explore the relationship between baseball performance and atmospheric air quality. The first step in this project is to import the multiple data sets with the goal of building a shared data frame so that the data can be easily compared using Pandas.


In [4]:
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
%matplotlib inline

#Set filepaths for baseball data and atmospheric data
team_path = "C:\Users\Nathan\Documents\GitHub\SabermetricsProject\Data\Teams.csv"

#epa_daily_path = "C:\Users\Nathan\Documents\GitHub\SabermetricsProject\Data\epa_hap_daily_summary.csv"
epa_yearly_path = "C:\Users\Nathan\Documents\GitHub\SabermetricsProject\Data\epa_air_quality_annual_summary.csv"


team_DF = pd.read_csv(team_path)
#epa_daily_DF = pd.read_csv(epa_daily_path)
epa_yearly_DF = pd.read_csv(epa_yearly_path)

#Edit dates to contain only year
epa_yearly_DF["date_of_last_change"] = epa_yearly_DF["date_of_last_change"].str[:4]
epa_yearly_DF.rename(columns={"date_of_last_change":"year"},inplace=True)

team_DF.rename(columns={"yearID":"year"},inplace=True)

print("Original dataframe size = %s" %epa_yearly_DF.size)
#print("Memory usage for EPA yearly summary = %s" % epa_yearly_DF.memory_usage)

Original dataframe size = 112129050


Now that the data has been imported, the next  step is to clean the data by removing unneeded information and then  try to determine a shared key/column so that the dataframes can be merged. The epa readings contained in the dataframe were collected at a variety of cities within the US, many without any professional baseball team. There also may be cities that have atmospheric readings but no MLB team. Therefore I will identify the cities contained within both datasets and use these for my analysis

In [6]:
#Create a list of distinct cities where readings were collected
distinct_cities = epa_yearly_DF.city_name.unique()

MLB_cities = ["Baltimore","Boston","Chicago","Cleveland","Detroit","Houston","Kansas City","Anaheim",
             "Minneapolis","New York","Oakland","Seattle","St. Petersburg","Arlington","Phoenix",
             "Atlanta","Cincinnati","Denver","Los Angeles","Miami","Milwaukee","Philadelphia",
             "Pittsburgh","San Diego","San Francisco","St. Louis","Washington","St. Paul","Newark",]

shared_cities = [x for x in distinct_cities if x in MLB_cities]

epa_yearly_DF = epa_yearly_DF[epa_yearly_DF['city_name'].isin(shared_cities)]

print("Reduced dataframe size = %s" %epa_yearly_DF.size)

#Create a dictionary mapping teams to cities
team_cities = team_DF.name.unique()
team_to_city = {}
for words in team_cities:
    wordsList = words.split(" ")
    if wordsList[0]=="Kansas" or wordsList[0]=="San" or wordsList[0]=="Los" or wordsList[0]=="New" or wordsList[0]=="St.":
        team_to_city[words] = wordsList[0]+" "+wordsList[1]
    else:
        team_to_city[words] = wordsList[0]

#Add new city_name column to team dataframe to act as key
team_DF["city_name"] = "default"


for index,row in team_DF.iterrows():
    if row["name"] in team_to_city:
        team_DF.set_value(index,"city_name",team_to_city[row["name"]])

merged_DF = pd.merge(team_DF,epa_yearly_DF,on="year")



Reduced dataframe size = 13353175


ValueError: Buffer has wrong number of dimensions (expected 1, got 2)